### Test wtih JuMP

In [4]:
using JuMP , Ipopt
include("../Problems/JuMP/space_Shuttle_JMP.jl")
SpaceShuttleJMP = space_Shuttle_JMP();

set_optimizer(SpaceShuttleJMP,Ipopt.Optimizer)
set_attribute(SpaceShuttleJMP, "print_level", 5)
set_optimizer_attribute(SpaceShuttleJMP,"tol",1e-8)
set_optimizer_attribute(SpaceShuttleJMP,"constr_viol_tol",1e-6)
set_optimizer_attribute(SpaceShuttleJMP,"max_iter",1000)

In [5]:
println("Solving...")
optimize!(SpaceShuttleJMP)
println()